# Comparison with ChatGPT <br>

The following Code was executed in <b>Google Colab</b>, since there were some compatability-problems with local jupyter. <br>
Here, I wanted to see how a LLM would do in the task - can ChatGPT differentiate between route instructions and sights - <br>
as I did before manually - and how good does it perform in such a special NER-task? 

The code used below as well as the additional jinja-file was provided by Nina Claudia Rastinger, ÖAW. 

In [1]:
import os
import sys

!pip install git+https://github.com/promptslab/Promptify.git
from promptify import Prompter, OpenAI, Pipeline

from google.colab import drive
drive.mount('/content/drive')

text = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/Sonntag_Teil.txt', 'r', encoding='utf8')
Sonntag_text = ' '.join(text)


In [2]:
api_key = 'your_key'

In [3]:
def prompter_ner(text):
    result = pipe.fit(text,
                      domain = "alte Skripte",
                      examples = one_shot,
                      labels = ["Ort_auf_Route", "Ort_im_Blickfeld"])
    return result#[0]["parsed"]["data"]["completion"][0]

In [ ]:
model = OpenAI(api_key)
prompter = Prompter('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/ner_adapted.jinja')
pipe = Pipeline(prompter, model)

In [ ]:
one_shot_text = "Geht man nun auf den Wall rechts (Südwest) am Palais des Erzherzogs Karl vorüber, so kömmt man zu der Auffahrt vom Spitalplatze, am Hofbau= amte vorbei, zum Kärnthnerthor=Theater, zu welchem ein bedeckter Gang vom Wall hinüber führt. Wenige Schritte weiter steht man über dem Kärnthnerthore und sieht in die Kärnthnerstraße hinab. Auf dem Glacis sieht man die Brücke über die Wien, welche zur Vorstadt Wieden, auf die Straße nach Baden führt. Rechts steht das große Starhembergische Freihaus, links das Polytech= nische Jnstitut und die Karlskirche."

In [4]:
one_shot = [[one_shot_text,
             [{"E": "Wall", "T": "Ort_auf_Route"},
              {"E": "Palais des Erzherzogs Karl", "T": "Ort_auf_Route"},
              {"E": "Spitalplatze", "T": "Ort_auf_Route"},
              {"E": "Hofbau= amte", "T": "Ort_auf_Route"},
              {"E": "Kärnthnerthor=Theater", "T": "Ort_im_Blickfeld"},
              {"E": "Kärthnerthore", "T": "Ort_auf_Route"},
              {"E": "Kärnthnerstraße", "T": "Ort_im_Blickfeld"},
              {"E": "Glacis", "T": "Ort_auf_Route"},
              {"E": "Brücke über die Wien", "T": "Ort_im_Blickfeld"},
              {"E": "Starhembergische Freihaus", "T": "Ort_im_Blickfeld"},
              {"E": "Polytech= nische Jnstitut", "T": "Ort_im_Blickfeld"},
              {"E": "Karlskirche", "T": "Ort_im_Blickfeld"}]]]

In [ ]:
part1_text = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/Sonntag_Teil.txt', 'r', encoding='utf8') #unpickling it 
Sonntag_1 = ' '.join(part1_text)  

In [5]:
result = prompter_ner(Sonntag_1) 

I repeated the above function 8 times, using always another bit of my text (I tried to do it with the whole data at once, filesize around 13 KB, but repeatedly got Invalid request oder Rate Limit error). <br>
For really validating the performance, you probably would have to repeat the while process a couple of times, seeing how good the model performs round after round. <br>
Since this was just a mere experiment, I refrained from that. <br>
But this could be another task for the future. 

In [ ]:
#for entity in result:
  #print(entity)

In [ ]:
import pickle
import pandas as pd

In [ ]:

## this process of course has to be repeated for each single result, in case you want to save them in between steps! 
with open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result_data.pkl', 'wb') as fp: #saving the returned object for later use
    pickle.dump(result, fp)

In [ ]:
def unpack_and_order(result):
  first = result[0]
  second = first['text']
  third = second[2:-2]

  data = eval(third)

  df = pd.DataFrame(columns=['Entity', 'Label'])

  rows = []
  for entry in data:
      if 'E' in entry and 'T' in entry:
          row = {'Entity': entry['E'], 'Label': entry['T']}
          rows.append(row)

  df_filled = pd.concat([df, pd.DataFrame(rows)])
  return df_filled

In [169]:
file1= open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result_data.pkl', 'rb')
result_1 = pickle.load(file1, encoding='utf8')

file2 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result2_data.pkl', 'rb')
result_2 = pickle.load(file2, encoding='utf8')

file3 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result3_data.pkl', 'rb')
result_3 = pickle.load(file3, encoding='utf8')

file4 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result4_data.pkl', 'rb')
result_4 = pickle.load(file4, encoding='utf8')

file5 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result5_data.pkl', 'rb')
result_5 = pickle.load(file5, encoding='utf8')

file6 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result6_data.pkl', 'rb')
result_6 = pickle.load(file6, encoding='utf8')

file7 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result7_data.pkl', 'rb')
result_7 = pickle.load(file7, encoding='utf8')

file8 = open('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result8_data.pkl', 'rb')
result_8 = pickle.load(file8, encoding='utf8')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/result_data.pkl'

In [ ]:
df_1 = unpack_and_order(result_1)
df_2 = unpack_and_order(result_2)
df_3 = unpack_and_order(result_3)
df_4 = unpack_and_order(result_4)
df_5 = unpack_and_order(result_5)
#df_6 = unpack_and_order(result_6)  #here, we had a /n error in our result
df_7 = unpack_and_order(result_7)
df_8 = unpack_and_order(result_8)

In [ ]:
#correction for sixth result
first = result_6[0]
second = first['text']
third = second[2:-2]

fourth = third.replace("\n", "")
data = eval(fourth)

df = pd.DataFrame(columns=['Entity', 'Label'])
rows = []
for entry in data:
    if 'E' in entry and 'T' in entry:
        row = {'Entity': entry['E'], 'Label': entry['T']}
        rows.append(row)

df_6 = pd.concat([df, pd.DataFrame(rows)])


In [ ]:
frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8]
full_df = pd.concat(frames)

In [ ]:
full_df.to_csv('/content/drive/MyDrive/Colab Notebooks/ReiseführerProjekt/full_df.csv', index=False)

Here, I got back to my local jupyter. You can find all the data & resources used in Golab in this file. <br>
<br>
Let's start by importing the full_df back into the kernel! 

In [55]:
import os 
import pandas as pd
import numpy as np


In [58]:
full_df = pd.read_csv('../ChatGPT_API/ReiseführerProjekt/full_df.csv')

In [59]:
print(full_df.head())

                          Entity             Label
0                        Ravelin     Ort_auf_Route
1  Palais des Herzogs von Koburg  Ort_im_Blickfeld
2            Courtine des Walles     Ort_auf_Route
3         orientalische Akademie  Ort_im_Blickfeld
4                     Stubenthor     Ort_auf_Route


In [38]:
print(len(full_df[full_df['Label'] == 'Ort_auf_Route']))
print(len(full_df[full_df['Label'] == 'Ort_im_Blickfeld']))

50
128


Since the task for ChatGPT was to identify the places that lay on the route as well as the places that were visible at the several parts of the route, I assigned two labels: place on a route, and place in sight. <br> 
At first glance, looking only at the numbers, the model didn't perform so bad: It found 178 places, 50 of them on the route, and 128 of them in sight - which seems to be a pretty sensible ratio!  
<br>
<br>
Now, we should see if we can match the places the model found with the places I manually extracted from the text, and at the same time add the coordinates we already found on our map. 

In [46]:
# import the dataset for sunday with all places & coordinates

coordinate_table = pd.read_excel('C:/Users/sarah/Desktop/git_folder/DataScienceProject/Data/Sonntag/joined_Sonntag_coord_nn.xlsx')

In [47]:
print(coordinate_table.head())

   Index                     NEUE_NAMEN              ADRESSE Sicherheit  \
0      0                     k. k. Burg                  NaN       hoch   
1      1                      Burgplatz                  NaN       hoch   
2      2                       Burgthor                  NaN       hoch   
3      3        kaiserlichen Stallungen                  NaN       hoch   
4      4  Gebäude der ungarischen Garde  07., Museumstraße 7     mittel   

   Longitude   Latitude                             Quelle  \
0  16.365809  48.206874  Gazetteer_Steinhausenplan_V5_auto   
1  16.364500  48.206794                 QGIS - Estimation    
2  16.362702  48.205781                 QGIS - Estimation    
3  16.358862  48.203593                 QGIS - Estimation    
4  16.355278  48.205833                 Wikipedia, GeoHack   

                                                 URL  
0                                                NaN  
1                                                NaN  
2          

In [48]:
# extract the first column as a list to do some fuzzy string matching

place_list = coordinate_table['NEUE_NAMEN'].tolist()

In [33]:
from thefuzz import process, fuzz 

In [60]:
full_df['MATCH_SCORE'] = full_df['Entity'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[1])
full_df['NEUE_NAMEN'] = full_df['Entity'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0])
full_df = full_df.sort_values('MATCH_SCORE', ascending=False)

In [61]:
full_df.head(50)

,Entity,Label,MATCH_SCORE,NEUE_NAMEN
0,Ravelin,Ort_auf_Route,100,Ravelin
126,Ferdinandsbrücke,Ort_auf_Route,100,Ferdinandsbrücke
123,Kahlengebirge,Ort_im_Blickfeld,100,Kahlengebirge
122,Donaukanal,Ort_auf_Route,100,Donaukanal
121,Ravelin,Ort_auf_Route,100,Ravelin
118,Fahrpost,Ort_auf_Route,100,Fahrpost
113,Jnvalidenhaus,Ort_im_Blickfeld,100,Jnvalidenhaus
112,Stückbohrerei,Ort_im_Blickfeld,100,Stückbohrerei
110,Münzgebäude,Ort_im_Blickfeld,100,Münzgebäude
105,Stubenthor,Ort_auf_Route,100,Stubenthor


In [62]:
full_df.tail(50)

,Entity,Label,MATCH_SCORE,NEUE_NAMEN
77,Neustädter Kanal,Ort_im_Blickfeld,77,Neustädter
111,Neustädter Kanal,Ort_im_Blickfeld,77,Neustädter
158,Ravelin Mölkerbastei,Ort_auf_Route,75,Mölkerbastei
98,Fischerkähne,Ort_im_Blickfeld,75,Fischerthore
165,geographische Jnstitut des Generalstabes,Ort_im_Blickfeld,71,geographische Jnstitut
116,hoher Ravelin,Ort_im_Blickfeld,70,Ravelin
61,kleiner Gar= ten des Erzherzogs Karl,Ort_im_Blickfeld,70,Palais des Erzherzogs Karl
19,Hofbiblio= thek=Gebäudes am Josephsplatze,Ort_im_Blickfeld,68,Hofbibliotek=Gebäudes
144,Dach des Arsenales,Ort_im_Blickfeld,67,Arsenales
129,Praterfahrt,Ort_im_Blickfeld,67,Praters


A reasonable limit would be 61, I propose - after that, there's not much to be matched correctly. <br>
Single exeptions like Hauptmauth can be added manually later on. <br> But from the look of it, only around 20-30 (from 178) seem to be kind of wrong, some definitely also the fault of the textsource the model had to work with. 


In [63]:
full_df.loc[full_df.MATCH_SCORE.between(0,60), 'NEUE_NAMEN'] = np.nan

In [64]:
print(full_df)

                                                Entity             Label  \
0                                              Ravelin     Ort_auf_Route   
126                                   Ferdinandsbrücke     Ort_auf_Route   
123                                      Kahlengebirge  Ort_im_Blickfeld   
122                                         Donaukanal     Ort_auf_Route   
121                                            Ravelin     Ort_auf_Route   
..                                                 ...               ...   
6           Steinbrücke über die Wienist Wien = Donau?  Ort_im_Blickfeld   
28   Reiterstatue des Gemahls von M. Theresia, Kais...  Ort_im_Blickfeld   
43                                         Flügelsalon  Ort_im_Blickfeld   
75   VorstadtVorstadt = Bezirk LandstraßeVorstadt =...     Ort_auf_Route   
7    VorstadtVorstadt = Bezirk LandstraßeVorstadt =...  Ort_im_Blickfeld   

     MATCH_SCORE        NEUE_NAMEN  
0            100           Ravelin  
126          

In [142]:
full_df.sort_index(axis=0, ascending=True, inplace=True) #recreate the correct order

In [143]:
full_df

,Entity,Label,MATCH_SCORE,NEUE_NAMEN
0,Ravelin,Ort_auf_Route,100,Ravelin
1,Palais des Herzogs von Koburg,Ort_im_Blickfeld,100,Palais des Herzogs von Koburg
2,Courtine des Walles,Ort_auf_Route,54,NaN
3,orientalische Akademie,Ort_im_Blickfeld,100,orientalische Akademie
4,Stubenthor,Ort_auf_Route,100,Stubenthor
...,...,...,...,...
173,Volksgarten,Ort_im_Blickfeld,100,Volksgarten
174,Schenkenstraße,Ort_im_Blickfeld,100,Schenkenstraße
175,Palais Liech= tenstein,Ort_im_Blickfeld,95,Palais Liechtenstein
176,doppelte Brücke,Ort_im_Blickfeld,62,große Steinbrücke


Now, let's combine both dataframes depending on the NEUE_NAMEN column in order to create our QGIS dataframe.

In [166]:
result = pd.merge(full_df, coordinate_table, on="NEUE_NAMEN", how="left")
result = result.drop(columns=["MATCH_SCORE", "Index"])

In [168]:
result.to_excel('C:/Users/sarah/Desktop/git_folder/DataScienceProject/Data/Sonntag/Chatgpt_coord.xlsx')

In [ ]:
## I had to delete some rows that were duplicated & manually added some missing coordinates
## also I saw that some parts were very similar, albeit this might have been my mistake - so better redo the process maybe
result_new = pd.read_excel('C:/Users/sarah/Desktop/git_folder/DataScienceProject/Data/Sonntag/Chatgpt_coord_nn.xlsx')
result_new.to_csv('C:/Users/sarah/Desktop/git_folder/DataScienceProject/Data/Sonntag/Chatgpt_coord_nn.csv')